In [ ]:
# Setup execution path and pf client

import os
import promptflow

root = os.path.join(os.getcwd(), "../")
flow_path = os.path.join(root, "named-entity-recognition")
data_path = os.path.join(flow_path, "data.jsonl")
eval_match_rate_flow_path = os.path.join(root, "../evaluation/eval-entity-match-rate")

pf = promptflow.PFClient()


In [ ]:
# Run flow against test data
run = pf.run(
    flow=flow_path,
    data=data_path,
    column_mapping={
        "text": "${data.text}",
        "entity_type": "${data.entity_type}"
    },
    display_name="ner_bulk_run",
    tags={"unittest": "true"},
    stream=True)

In [ ]:
# Show output of flow run
pf.get_details(run)

In [ ]:
# Evaluate the match rate of the entity recognition result of the flow run
eval = pf.run(
    flow=eval_match_rate_flow_path,
    run=run,
    data=data_path,
    column_mapping={
        "entities": "${run.outputs.entities}",
        "ground_truth": "${data.results}"
    },
    display_name="eval_match_rate",
    tags={"unittest": "true"},
    stream=True)

pf.get_details(eval)

In [ ]:
# Get metrics of the evaluation flow run
pf.get_metrics(eval)

In [ ]:
# Visualize the flow run and evaluation run with HTML
pf.visualize([run, eval])